## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,80.33,71,20,9.22,light rain
1,1,Vanimo,PG,-2.6741,141.3028,77.13,90,67,6.82,broken clouds
2,2,Kimbe,PG,-5.5502,150.1429,74.23,92,11,3.85,few clouds
3,3,Ruidoso,US,33.3317,-105.6730,63.10,94,90,11.50,overcast clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,40.23,98,100,4.88,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,80.33,71,20,9.22,light rain
1,1,Vanimo,PG,-2.6741,141.3028,77.13,90,67,6.82,broken clouds
16,16,Kavieng,PG,-2.5744,150.7967,82.33,78,9,15.01,light rain
17,17,Ushirombo,TZ,-3.4919,31.9639,75.70,36,91,0.78,overcast clouds
21,21,Albany,US,42.6001,-73.9662,81.14,50,36,1.99,scattered clouds
26,26,Carutapera,BR,-1.1950,-46.0200,80.56,83,91,7.58,overcast clouds
27,27,Suez,EG,29.9737,32.5263,81.21,60,0,11.70,clear sky
30,30,Hithadhoo,MV,-0.6000,73.0833,82.42,72,100,14.70,overcast clouds
32,32,Puerto Ayora,EC,-0.7393,-90.3518,78.76,75,72,10.38,broken clouds
35,35,Muli,MV,2.9167,73.5667,82.20,75,100,17.96,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.33,light rain,19.7297,-155.0900,
1,Vanimo,PG,77.13,broken clouds,-2.6741,141.3028,
16,Kavieng,PG,82.33,light rain,-2.5744,150.7967,
17,Ushirombo,TZ,75.70,overcast clouds,-3.4919,31.9639,
21,Albany,US,81.14,scattered clouds,42.6001,-73.9662,
26,Carutapera,BR,80.56,overcast clouds,-1.1950,-46.0200,
27,Suez,EG,81.21,clear sky,29.9737,32.5263,
30,Hithadhoo,MV,82.42,overcast clouds,-0.6000,73.0833,
32,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
35,Muli,MV,82.20,overcast clouds,2.9167,73.5667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
output_data_fileX = "Vacation_Search/WeatherPy_vacationX.csv"
hotel_df.to_csv(output_data_fileX, index_label="City_ID")

In [10]:
hotel_dfX = pd.read_csv("Vacation_Search/WeatherPy_vacationX.csv")
hotel_dfX.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
1,1,Vanimo,PG,77.13,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
2,16,Kavieng,PG,82.33,light rain,-2.5744,150.7967,Nusa Island Retreat
3,17,Ushirombo,TZ,75.70,overcast clouds,-3.4919,31.9639,Kigali Hotel
4,21,Albany,US,81.14,scattered clouds,42.6001,-73.9662,NaN


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_dfX.dropna()
clean_hotel_df = clean_hotel_df.set_index("City_ID")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
0,Hilo,US,80.33,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
1,Vanimo,PG,77.13,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
16,Kavieng,PG,82.33,light rain,-2.5744,150.7967,Nusa Island Retreat
17,Ushirombo,TZ,75.70,overcast clouds,-3.4919,31.9639,Kigali Hotel
26,Carutapera,BR,80.56,overcast clouds,-1.1950,-46.0200,Lidera
27,Suez,EG,81.21,clear sky,29.9737,32.5263,Green House
30,Hithadhoo,MV,82.42,overcast clouds,-0.6000,73.0833,Scoop Guest House
32,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
35,Muli,MV,82.20,overcast clouds,2.9167,73.5667,Medhufushi Island Resort


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

KeyError: 'Max temp'

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.50)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))